In [1]:
pip install python-dotenv



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv('API_KEY')
print(api_key)

24DD4E29-B6FF-31DC-9066-43A4107B0C42


In [2]:
import requests
import pandas as pd
base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'

def fetch_data(year):
    # states = ['CA', 'CO', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'ME', 'MD', 'MI',
    #    'MN', 'MO', 'MT', 'NE', 'NV', 'NM', 'NY', 'NC', 'ND', 'OH', 'OR',
    #    'PA', 'SD', 'UT', 'VA', 'WA', 'WI', 'WY', 'VT', 'MA', 'TX', 'AZ',
    #    'AR', 'DE', 'FL', 'GA', 'LA', 'NJ', 'OK', 'RI', 'SC', 'TN', 'WV',
    #    'CT', 'MS', 'AL', 'HI', 'NH', 'AK']
    
    statistics = ['AREA BEARING', 'AREA HARVESTED']
    total_df = pd.DataFrame()
    
    
    # for state in states:
    for stat in statistics:
        field_crop_params = {
            'key': api_key,
            'source_desc': 'CENSUS',
            'sector_desc': 'CROPS',
            'statisticcat_desc': stat,
            'year': year,
            'agg_level_desc': 'STATE',
            'freq_desc': 'Annual',
            'format': 'JSON',
            'unit_desc': 'ACRES',
            'domain_desc': 'TOTAL',
            # 'prodn_practice_desc':'ALL PRODUCTION PRACTICES'
        }
        response = requests.get(base_url, params=field_crop_params)
        
        if response.status_code == 200:
            raw = response.json()
            df = pd.DataFrame(raw['data'])
            total_df = pd.concat([df, total_df], ignore_index=True)
        else:
            print(f"Error fetching data for  - {stat}: {response.status_code}")

    return total_df

    
    
#stdrvmkcr


In [21]:
def normalize_data(data):
    total_df = data
    filtered_df = total_df[total_df['short_desc'].str.contains('ACRES HARVESTED', na=False)]
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FRUIT TOTALS', 'FRUIT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FRUIT, OTHER', 'FRUIT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUT TOTALS', 'TREE NUT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUTS, OTHER', 'TREE NUT')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('BERRIES, OTHER', 'BERRIES')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('BERRY TOTALS', 'BERRIES')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FIELD CROP', 'FIELD CROPS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FIELD CROP TOTALS', 'FIELD CROPS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('FIELD CROPS, OTHER', 'FIELD CROPS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('CITRUS TOTALS', 'CITRUS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUT TOTALS', 'TREE NUTS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUTS, OTHER', 'TREE NUTS')  
    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].str.replace('TREE NUTS, OTHER', 'TREE NUTS')  


    filtered_df['Value'] = filtered_df['Value'].replace({'(D)': np.nan}, regex=True)  # Replace (D) with NaN
    filtered_df['Value'] = filtered_df['Value'].str.replace(',', '')  # Remove commas
    filtered_df['Value'] = pd.to_numeric(filtered_df['Value'], errors='coerce')  # Convert to numeric, invalid parsing will become NaN
    agreggate = filtered_df.copy()
    agreggate = pd.DataFrame(agreggate)
    
    def normalize_commodity(row):
    # Define a list of commodities to normalize
        commodities_to_normalize =agreggate['commodity_desc'].unique()
        
        # Loop through each commodity and check if it exists in the row's 'Commodity' column
        for commodity in commodities_to_normalize:
            if commodity in row['commodity_desc']:
                return commodity
        
        return row['commodity_desc']
    # Apply the normalization function to the 'Commodity' column
    agreggate['commodity_desc'] = agreggate.apply(normalize_commodity, axis=1)
    # Group by relevant columns and sum the values
    aggregated_df = (
        agreggate.groupby(['state_name', 'commodity_desc'], as_index=False)
        .agg({'Value': 'sum'})
    )

    # Pivot the table to have commodities as columns
    pivot_df = aggregated_df.pivot(index='state_name', columns='commodity_desc', values='Value').reset_index()

    pivot_df = pivot_df.fillna(0)

    return pivot_df
    

In [3]:
def faster_normalize_data(data):
    # strictly better version, the one above leaves out key features and is so slow

    # crop_lifecycle = {
    #     'GRAPES': 'perennial',
    #     'APPLES': 'perennial',
    #     'RASPBERRIES': 'perennial',
    #     'PLUMS': 'perennial',
    #     'BLACKBERRIES': 'perennial',
    #     'PEACHES': 'perennial',
    #     'CHERRIES': 'perennial',
    #     'WHEAT': 'annual',
    #     'OLIVES': 'perennial',
    #     'BROCCOLI': 'annual',
    #     'SUNFLOWER': 'annual',
    #     'LEMONS': 'perennial',
    #     'BEANS': 'annual',
    #     'AVOCADOS': 'perennial',
    #     'POTATOES': 'annual',
    #     'CORN': 'annual',
    #     'ORANGES': 'perennial',
    #     'ALMONDS': 'perennial',
    #     'SPINACH': 'annual',
    #     'LETTUCE': 'annual',
    #     'PISTACHIOS': 'perennial',
    #     'PRUNES': 'perennial',
    #     'TOMATOES': 'annual',
    #     'PEARS': 'perennial',
    #     'SORGHUM': 'annual',
    #     'WALNUTS': 'perennial',
    #     'RICE': 'annual'
    # }
    # data['lifecycle'] = data['commodity_desc'].map(crop_lifecycle)

    # Filter: keep AREA BEARING for perennials, AREA HARVESTED for annuals
    condition = (
        ((data['lifecycle'] == 'perennial') & (data['statisticcat_desc'] == 'AREA BEARING')) |
        ((data['lifecycle'] == 'annual') & (data['statisticcat_desc'] == 'AREA HARVESTED'))
    )
    
    filtered_df = data[condition]

    # Normalize commodity descriptions using a mapping dictionary
    replacements = {
        'FRUIT TOTALS': 'FRUIT',
        'FRUIT, OTHER': 'FRUIT',
        'TREE NUT TOTALS': 'TREE NUTS',
        'TREE NUTS, OTHER': 'TREE NUTS',
        'BERRIES, OTHER': 'BERRIES',
        'BERRY TOTALS': 'BERRIES',
        'FIELD CROP': 'FIELD CROPS',
        'FIELD CROP TOTALS': 'FIELD CROPS',
        'FIELD CROPS, OTHER': 'FIELD CROPS',
        'CITRUS TOTALS': 'CITRUS'
    }

    filtered_df['commodity_desc'] = filtered_df['commodity_desc'].replace(replacements)
    
    filtered_df['Value'] = (
        filtered_df['Value']
        .replace({'(D)': np.nan}, regex=True)
        .str.replace(',', '', regex=False)
    )
    filtered_df['Value'] = pd.to_numeric(filtered_df['Value'], errors='coerce')

    # Create standardized 'target_desc': "COMMODITY - ACRES HARVESTED"
    filtered_df['target_desc'] = filtered_df['commodity_desc'].str.upper() + ' - ACRES HARVESTED'

    # Create two masks:
    # 1. Direct matches: short_desc matches "COMMODITY - ACRES HARVESTED"
    direct_match = filtered_df[filtered_df['short_desc'] == filtered_df['target_desc']]

    # 2. Fallback matches: those without a direct match, we sum grouped by commodity/state
    all_states = filtered_df['state_name'].unique()
    all_commodities = filtered_df['commodity_desc'].unique()

    # Set for faster lookup
    direct_keys = set(zip(direct_match['state_name'], direct_match['commodity_desc']))

    # Fallback data: those not already in direct_match
    fallback_data = filtered_df[
        ~filtered_df.set_index(['state_name', 'commodity_desc']).index.isin(direct_keys)
    ]

    fallback_agg = fallback_data.groupby(['state_name', 'commodity_desc'], as_index=False)['Value'].sum()

    # Combine both direct and fallback results
    direct_agg = direct_match.groupby(['state_name', 'commodity_desc'], as_index=False)['Value'].sum()
    combined = pd.concat([direct_agg, fallback_agg])

    # Remove duplicates, keeping direct match where possible
    combined = combined.sort_values(by='Value', ascending=False).drop_duplicates(subset=['state_name', 'commodity_desc'])

    # Pivot to final format: 50 states x commodities
    final_df = combined.pivot(index='state_name', columns='commodity_desc', values='Value').fillna(0).reset_index()

    return final_df

In [ ]:
for i in range(2012, 2025):
    data = fetch_data(i)

    if data is not None and not data.empty:
        print(f"{i} sucess")
        data.to_csv(f"{i}full_raw_api.csv")
        
        norm = faster_normalize_data(data)
        norm.to_csv(f"norm-data/{i}_raw_api.csv")

        
    else:
        print(f'{i} failure')

2012 sucess


/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['commodity_desc'] = filtered_df['commodity_desc'].replace(replacements)
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Value'] = (
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy

Error fetching data for  - AREA BEARING: 400
2013 sucess


/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['commodity_desc'] = filtered_df['commodity_desc'].replace(replacements)
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Value'] = (
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy

Error fetching data for  - AREA BEARING: 400
2014 sucess
Error fetching data for  - AREA BEARING: 400
Error fetching data for  - AREA HARVESTED: 400
2015 failure
Error fetching data for  - AREA BEARING: 400
Error fetching data for  - AREA HARVESTED: 400
2016 failure
2017 sucess


...

Error fetching data for  - AREA BEARING: 400
2019 sucess
Error fetching data for  - AREA BEARING: 400
Error fetching data for  - AREA HARVESTED: 400
2020 failure
Error fetching data for  - AREA BEARING: 400
Error fetching data for  - AREA HARVESTED: 400
2021 failure
2022 sucess


/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['commodity_desc'] = filtered_df['commodity_desc'].replace(replacements)
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Value'] = (
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy

Error fetching data for  - AREA BEARING: 400
2023 sucess
Error fetching data for  - AREA BEARING: 400


/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['commodity_desc'] = filtered_df['commodity_desc'].replace(replacements)
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Value'] = (
/var/folders/xj/3_cf3p513dxgf8tbpc4j_1j40000gn/T/ipykernel_31876/3409413826.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy

Error fetching data for  - AREA HARVESTED: 400
2024 failure
